In [5]:
import os
os.environ["JAVA_HOME"] = "/opt/homebrew/opt/openjdk@17/libexec/openjdk.jdk/Contents/Home"
print(os.environ["JAVA_HOME"])

/opt/homebrew/opt/openjdk@17/libexec/openjdk.jdk/Contents/Home


In [17]:
!/usr/libexec/java_home -V

Matching Java Virtual Machines (3):
    24.0.1 (arm64) "Oracle Corporation" - "Java SE 24.0.1" /Library/Java/JavaVirtualMachines/jdk-24.jdk/Contents/Home
    21.0.7 (arm64) "Homebrew" - "OpenJDK 21.0.7" /opt/homebrew/Cellar/openjdk@21/21.0.7/libexec/openjdk.jdk/Contents/Home
    11.0.27 (arm64) "Homebrew" - "OpenJDK 11.0.27" /opt/homebrew/Cellar/openjdk@11/11.0.27/libexec/openjdk.jdk/Contents/Home
/Library/Java/JavaVirtualMachines/jdk-24.jdk/Contents/Home


In [ ]:
#Step: 1 - Load Dataset in Jupyter Notebook#

In [41]:
!pip install spark

In [43]:
!pip install pyspark

In [1]:
import pandas as pd
df = pd.read_csv('/Users/akbarbasha/Desktop/Customer mall projects/Mall_Customers.csv')
print(df)

     CustomerID   Genre  Age  Annual Income (k$)  Spending Score (1-100)
0             1    Male   19                  15                      39
1             2    Male   21                  15                      81
2             3  Female   20                  16                       6
3             4  Female   23                  16                      77
4             5  Female   31                  17                      40
..          ...     ...  ...                 ...                     ...
195         196  Female   35                 120                      79
196         197  Female   45                 126                      28
197         198    Male   32                 126                      74
198         199    Male   32                 137                      18
199         200    Male   30                 137                      83

[200 rows x 5 columns]


In [ ]:
#step:2:start pyspark session #Before setting up spark. we need verify if spark java version is set to 17 version
#setting java variable explicitly to Java 11/17 as it is compatible with spark4.0 session #

In [7]:
import os 
print(os.environ.get("JAVA_HOME"))

/opt/homebrew/opt/openjdk@17/libexec/openjdk.jdk/Contents/Home


In [ ]:
#Confirming and saving the jupyter notebook java variable to 11 version

In [21]:
!$JAVA_HOME/bin/java -version

openjdk version "17.0.15" 2025-04-15
OpenJDK Runtime Environment Homebrew (build 17.0.15+0)
OpenJDK 64-Bit Server VM Homebrew (build 17.0.15+0, mixed mode, sharing)


In [ ]:
# System wide java version which is default java version set up of the systems #

In [51]:
!java -version

openjdk version "21.0.7" 2025-04-15
OpenJDK Runtime Environment Homebrew (build 21.0.7)
OpenJDK 64-Bit Server VM Homebrew (build 21.0.7, mixed mode, sharing)


In [ ]:
#step:3 -Initalzie spark session 

In [9]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("MallCustomerAnalysis") \
    .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/09/12 19:21:12 WARN Utils: Your hostname, Akbars-MacBook-Air.local, resolves to a loopback address: 127.0.0.1; using 192.168.1.6 instead (on interface en0)
25/09/12 19:21:12 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/09/12 19:21:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [11]:
spark_df = spark.read.csv("/Users/akbarbasha/Desktop/Customer mall projects/mall_customers.csv", header=True, inferSchema=True)
spark_df.printSchema()

root
 |-- CustomerID: integer (nullable = true)
 |-- Genre: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Annual Income (k$): integer (nullable = true)
 |-- Spending Score (1-100): integer (nullable = true)



In [13]:
spark_df.show(5)

+----------+------+---+------------------+----------------------+
|CustomerID| Genre|Age|Annual Income (k$)|Spending Score (1-100)|
+----------+------+---+------------------+----------------------+
|         1|  Male| 19|                15|                    39|
|         2|  Male| 21|                15|                    81|
|         3|Female| 20|                16|                     6|
|         4|Female| 23|                16|                    77|
|         5|Female| 31|                17|                    40|
+----------+------+---+------------------+----------------------+
only showing top 5 rows


In [85]:
#Data Cleaning Steps (PySpark)#

#1.Verify datatypes
spark_df.printSchema()

#2.Rename/Normalize Column names
spark_df.withColumnRenamed("Annual Income (k$)", "Annual_Income") \
        .withColumnRenamed("Spending Score (1-100)", "Spending_Score")

#3.#Show rows with any isNull()/isNotNull values
from pyspark.sql.functions import col
spark_df.filter(spark_df["Genre"].isNotNull()).show()
spark_df.filter(spark_df["Genre"].isNull()).show()

#4.#Count rows with nulls in a specific column
spark_df.filter(col("Age").isNull()).count()

#5.Fill missing values
spark_df = spark_df.na.fill({"Age": 0})

#6.drop duplicates based on specific columns:
spark_df = spark_df.dropDuplicates(["CustomerID"])

root
 |-- CustomerID: integer (nullable = true)
 |-- Genre: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Annual Income (k$): integer (nullable = true)
 |-- Spending Score (1-100): integer (nullable = true)

+----------+------+---+------------------+----------------------+
|CustomerID| Genre|Age|Annual Income (k$)|Spending Score (1-100)|
+----------+------+---+------------------+----------------------+
|         1|  Male| 19|                15|                    39|
|         2|  Male| 21|                15|                    81|
|         3|Female| 20|                16|                     6|
|         4|Female| 23|                16|                    77|
|         5|Female| 31|                17|                    40|
|         6|Female| 22|                17|                    76|
|         7|Female| 35|                18|                     6|
|         8|Female| 23|                18|                    94|
|         9|  Male| 64|                19|    

0

In [87]:
#Descriptive Statstics#
spark_df.describe().show()

25/09/11 22:59:17 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+------------------+------+-----------------+------------------+----------------------+
|summary|        CustomerID| Genre|              Age|Annual Income (k$)|Spending Score (1-100)|
+-------+------------------+------+-----------------+------------------+----------------------+
|  count|               200|   200|              200|               200|                   200|
|   mean|             100.5|  NULL|            38.85|             60.56|                  50.2|
| stddev|57.879184513951124|  NULL|13.96900733155888| 26.26472116527124|    25.823521668370173|
|    min|                 1|Female|               18|                15|                     1|
|    max|               200|  Male|               70|               137|                    99|
+-------+------------------+------+-----------------+------------------+----------------------+



In [55]:
#PySpark Data Transformations#
from pyspark.sql import functions as F
from pyspark.sql.window import Window
#1.Filter transformation.
spark_df.filter(spark_df.Age>40).show()

+----------+------+---+------------------+----------------------+
|CustomerID| Genre|Age|Annual Income (k$)|Spending Score (1-100)|
+----------+------+---+------------------+----------------------+
|         9|  Male| 64|                19|                     3|
|        11|  Male| 67|                19|                    14|
|        13|Female| 58|                20|                    15|
|        19|  Male| 52|                23|                    29|
|        23|Female| 46|                25|                     5|
|        25|Female| 54|                28|                    14|
|        27|Female| 45|                28|                    32|
|        31|  Male| 60|                30|                     4|
|        33|  Male| 53|                33|                     4|
|        35|Female| 49|                33|                    14|
|        37|Female| 42|                34|                    17|
|        41|Female| 65|                38|                    35|
|        4

In [59]:
#2.Sorting
spark_df.orderBy("Age", "Annual Income (k$)").show()

+----------+------+---+------------------+----------------------+
|CustomerID| Genre|Age|Annual Income (k$)|Spending Score (1-100)|
+----------+------+---+------------------+----------------------+
|        34|  Male| 18|                33|                    92|
|        66|  Male| 18|                48|                    59|
|        92|  Male| 18|                59|                    41|
|       115|Female| 18|                65|                    48|
|         1|  Male| 19|                15|                    39|
|        62|  Male| 19|                46|                    55|
|        69|  Male| 19|                48|                    59|
|       112|Female| 19|                63|                    54|
|       114|  Male| 19|                64|                    46|
|       116|Female| 19|                65|                    50|
|       139|  Male| 19|                74|                    10|
|       163|  Male| 19|                81|                     5|
|         

In [43]:
#3.Show distinct values
spark_df.select("Annual Income (k$)").distinct().count()

64

In [41]:
#4.Limit function to show top records
spark_df.orderBy(F.col("Annual Income (k$)").desc()).limit(5).show(10)

+----------+------+---+------------------+----------------------+
|CustomerID| Genre|Age|Annual Income (k$)|Spending Score (1-100)|
+----------+------+---+------------------+----------------------+
|       200|  Male| 30|               137|                    83|
|       199|  Male| 32|               137|                    18|
|       197|Female| 45|               126|                    28|
|       198|  Male| 32|               126|                    74|
|       195|Female| 47|               120|                    16|
+----------+------+---+------------------+----------------------+



In [67]:
#5.Groupby and Aggregaton of Gender+Age
spark_df.groupBy("Genre").agg(F.max("Age").alias("MaxAge")).show()

+------+------+
| Genre|MaxAge|
+------+------+
|Female|    68|
|  Male|    70|
+------+------+



'''

Analysis report on the basis of the spending score 

'''

In [69]:
#1.summary statstics od spending score#
spark_df.select("Spending Score (1-100)").describe().show()

+-------+----------------------+
|summary|Spending Score (1-100)|
+-------+----------------------+
|  count|                   200|
|   mean|                  50.2|
| stddev|    25.823521668370173|
|    min|                     1|
|    max|                    99|
+-------+----------------------+



In [17]:
#2.Categorize customers based on the spends
from pyspark.sql.functions import when, col
spark_df = spark_df.withColumn("Spending_Category",
                               when(col("Spending Score (1-100)") <= 40, "Low Spender")
                               .when((col("Spending Score (1-100)") > 40) & (col("Spending Score (1-100)") <= 70), "Medium Spender")
                                .otherwise("High Spender")
                              )

In [19]:
spark_df.groupBy("Spending_Category").count().show()

+-----------------+-----+
|Spending_Category|count|
+-----------------+-----+
|   Medium Spender|   83|
|     High Spender|   54|
|      Low Spender|   63|
+-----------------+-----+



In [21]:
#3.Spending score by Gender
spark_df.groupBy("Genre").avg("Spending Score (1-100)").show()

+------+---------------------------+
| Genre|avg(Spending Score (1-100))|
+------+---------------------------+
|Female|         51.526785714285715|
|  Male|          48.51136363636363|
+------+---------------------------+



In [31]:
#4.Top 5 Average spending score per Income#
spark_df.groupBy("Annual Income (k$)").avg("Spending Score (1-100)").orderBy("Annual Income (k$)").show(5)

+------------------+---------------------------+
|Annual Income (k$)|avg(Spending Score (1-100))|
+------------------+---------------------------+
|                15|                       60.0|
|                16|                       41.5|
|                17|                       58.0|
|                18|                       50.0|
|                19|                       47.0|
+------------------+---------------------------+
only showing top 5 rows


In [55]:
#5.Avergae spending score as per Age
spark_df.groupBy("Age").avg("Spending Score (1-100)").orderBy("Age").show()

+---+---------------------------+
|Age|avg(Spending Score (1-100))|
+---+---------------------------+
| 18|                       60.0|
| 19|                      39.75|
| 20|                       40.2|
| 21|                       66.8|
| 22|                       70.0|
| 23|         63.333333333333336|
| 24|                       71.5|
| 25|         39.666666666666664|
| 26|                       54.5|
| 27|         60.333333333333336|
| 28|                       70.0|
| 29|                       76.6|
| 30|          80.28571428571429|
| 31|                     63.875|
| 32|                       66.0|
| 33|         54.333333333333336|
| 34|                       39.2|
| 35|         63.888888888888886|
| 36|                       52.5|
| 37|         15.333333333333334|
+---+---------------------------+
only showing top 20 rows


In [79]:
spark_df.select("Age").distinct().count()

51

In [81]:
#6.Top 10 customers by spending score
from pyspark.sql import functions as F
spark_df.orderBy(F.desc("Spending Score (1-100)")).limit(10).show()

+----------+------+---+------------------+----------------------+-----------------+
|CustomerID| Genre|Age|Annual Income (k$)|Spending Score (1-100)|Spending_Category|
+----------+------+---+------------------+----------------------+-----------------+
|        12|Female| 35|                19|                    99|     High Spender|
|        20|Female| 35|                23|                    98|     High Spender|
|       146|  Male| 28|                77|                    97|     High Spender|
|       186|  Male| 30|                99|                    97|     High Spender|
|       128|  Male| 40|                71|                    95|     High Spender|
|       168|Female| 33|                86|                    95|     High Spender|
|         8|Female| 23|                18|                    94|     High Spender|
|       142|  Male| 32|                75|                    93|     High Spender|
|       164|Female| 31|                81|                    93|     High S

In [83]:
#7.Top spender per Age group
spark_df.groupBy("Age").agg(F.max("Spending Score (1-100)").alias("Top_Score")).orderBy("Age").show()

+---+---------+
|Age|Top_Score|
+---+---------+
| 18|       92|
| 19|       59|
| 20|       75|
| 21|       81|
| 22|       79|
| 23|       94|
| 24|       92|
| 25|       73|
| 26|       55|
| 27|       89|
| 28|       97|
| 29|       88|
| 30|       97|
| 31|       93|
| 32|       93|
| 33|       95|
| 34|       90|
| 35|       99|
| 36|       92|
| 37|       32|
+---+---------+
only showing top 20 rows


In [85]:
#8.Top 10 customers by spending score
spark_df.orderBy(F.desc("Spending Score (1-100)")).limit(10).show()

+----------+------+---+------------------+----------------------+-----------------+
|CustomerID| Genre|Age|Annual Income (k$)|Spending Score (1-100)|Spending_Category|
+----------+------+---+------------------+----------------------+-----------------+
|        12|Female| 35|                19|                    99|     High Spender|
|        20|Female| 35|                23|                    98|     High Spender|
|       146|  Male| 28|                77|                    97|     High Spender|
|       186|  Male| 30|                99|                    97|     High Spender|
|       128|  Male| 40|                71|                    95|     High Spender|
|       168|Female| 33|                86|                    95|     High Spender|
|         8|Female| 23|                18|                    94|     High Spender|
|       142|  Male| 32|                75|                    93|     High Spender|
|       164|Female| 31|                81|                    93|     High S

In [87]:
#9.Maximum & Minimum spending score by Age
spark_df.groupBy("Age").agg(F.max("Spending Score (1-100)").alias("Max_Score"), F.min("Spending Score (1-100)").alias("Min_Score")).show()

+---+---------+---------+
|Age|Max_Score|Min_Score|
+---+---------+---------+
| 31|       93|       40|
| 65|       52|       35|
| 53|       46|        4|
| 34|       90|        1|
| 28|       97|       40|
| 27|       89|       47|
| 26|       55|       54|
| 44|       20|        7|
| 22|       79|       55|
| 47|       47|        9|
| 52|       29|       13|
| 20|       75|        5|
| 40|       95|       13|
| 57|       51|        5|
| 54|       59|       14|
| 48|       49|       36|
| 19|       59|        5|
| 64|        3|        3|
| 41|       39|       17|
| 43|       50|       17|
+---+---------+---------+
only showing top 20 rows


In [93]:
#10.Find correlation between Spending Score and Age
correlation = spark_df.stat.corr("Age", "Spending Score (1-100)")
print(f"Correlation between Age and Spending Score: {correlation}")

Correlation between Age and Spending Score: -0.32722684603909025
